# Flyer Basics

We want to understand how to build a *Flyer* in "BlueSky" to support various types of fly scans and remote data loggers.  The data about Flyers is spread about the standard documentation.  We need some clarity and a few examples that build complexity incrementally.

The basic notion of a Flyer is that it directs an *external controller* (we'll call it the *controller* here) to perform some data colelction process.  Usually, a *controller* is used to collect data at rates beyond the capabilities of BlueSky plans and the RunEngine.  Examples could be waveforms from a storage oscilloscope or a continuous motion scan of a diffractometer.

This notebook will show the basic requirements for a Flyer and build a simple working example you can use a template for your own work.  (Our inspiration to create this [basic flyer tutorial](https://github.com/prjemian/ipython_mintvm/issues/3#issuecomment-385058577) was the goal of operating various fly scans at the APS, such as the [USAXS fly scan](https://github.com/APS-USAXS/ipython-usaxs/issues/3), from BlueSky.  The working examples quickly became too instrument-specific to serve as tutorials.)


## Python imports and definitions <a class="anchor" id="imports" />

Here are the full set of packages to imported.  The first block are Python standard packages, then come the ophyd, BluSky, and databroker packages.  Just the parts we plan on using here.  Since this is also a tutorial, we will not rename imports or use other such shortcuts in the documentation (the online code has some shortcuts).

* Create a logger instance in case we want to investigate internal details as our code runs.
* Create an instance of the BlueSky RunEngine.
* Create an instance of the databroker using the `mongodb_config.yml` file on the local machine
* Arrange for the databroker to receive all events from the RunEngine


In [26]:
import logging
import threading
import time

import ophyd
import bluesky
import bluesky.plans
import databroker

logger = logging.getLogger()
RE = bluesky.RunEngine({})
db = databroker.Broker.from_config(databroker.temp_config())
RE.subscribe(db.insert)

0

### Bare Minimum Requirements for a Flyer <a class="anchor" id="flyer-requirements" />

In BlueSky, a [Flyer](http://nsls-ii.github.io/bluesky/async.html?highlight=flyer#flying) is an [`ophyd.Device`](http://nsls-ii.github.io/ophyd/device-overview.html) that meets the [Fly-able interface](http://nsls-ii.github.io/ophyd/architecture.html#fly-able-interface), which has three methods:

1. Kickoff - begin accumulating data
1. Complete - BlueSky tells the Flyer that BlueSky is ready to receive data
1. Collect - the device provides the data to BlueSky

The first two methods [must return](http://nsls-ii.github.io/bluesky/hardware.html#kickoff) an instance of [`ophyd.status.Status`](http://nsls-ii.github.io/ophyd/generated/ophyd.status.Status.html) (a.k.a. a *status* object).  For our code, we choose a specialized version of `Status()`, the [`DeviceStatus()`](http://nsls-ii.github.io/ophyd/generated/ophyd.status.DeviceStatus.html) object, based on the [explanation provided in discussion](https://github.com/NSLS-II/tutorial-docker/pull/7#discussion_r185054364).

The `collect()` method requires a companion `describe_collect()` that informs the RunEngine what kind of data to expect from `collect()`.  If no timestamp information is provided from the *controller*, then do as we show here and use the workstation's clock to provide a timestamp for the event.

This example (which does absolutely nothing) meets the bare minimum requirement.

In [2]:
class BareMinimumFlyer(ophyd.Device):

    def kickoff(self):
        kickoff_status = ophyd.DeviceStatus(self)
        kickoff_status._finished(success=True)
        return kickoff_status

    def complete(self):
        complete_status = ophyd.DeviceStatus(self)
        complete_status._finished(success=True)
        return complete_status

    def collect(self):
        yield {'data':{}, 'timestamps':{}, 'time':time.time()}
    
    def describe_collect(self):
        return {self.name: {}}


flyer = BareMinimumFlyer(name="flyer")
print(flyer.complete())
print(list(flyer.collect()))

# if this next step succeeds, it's proof that we did this right!
RE(bluesky.plans.fly([flyer]))

Status(obj=BareMinimumFlyer(prefix='', name='flyer', read_attrs=[], configuration_attrs=[]), done=True, success=True)
[{'data': {}, 'timestamps': {}, 'time': 1525108857.5397317}]


('e66ee09c-05d7-43d8-b783-67b0d21bda2b',)

Only two `documents` were emitted from the `RunEngine`: `start` and `stop` since we generated no data content.

In [3]:
list(db[-1].documents())

[('start',
  {'plan_name': 'fly',
   'plan_type': 'generator',
   'scan_id': 1,
   'time': 1525108857.544967,
   'uid': 'e66ee09c-05d7-43d8-b783-67b0d21bda2b'}),
 ('stop',
  {'exit_status': 'success',
   'num_events': {'flyer': 1},
   'run_start': 'e66ee09c-05d7-43d8-b783-67b0d21bda2b',
   'time': 1525108857.57242,
   'uid': '84890c26-d006-472b-8e96-c6016265dcc5'})]

### Flyer : a starting template <a class="anchor" id="flyer-template" />

The `BareMinimumFlyer` is a good start to use a Flyer but we'll need to add a few more things to make a good template.  The first thing to do is to make the status object known to any method of the class.  We'll call it `self._completion_status` and it will tell us if the *controller* is finished.  In the constructor (`__init__()`), we set it to `None`, the value we expect when not *flying*.  Since we **need** a constructor, we must remember to call the constructor of the superclass as well or our `ophyd.Device` will not work correctly.

    def __init__(self, *args, **kwargs):
        super().__init__('', parent=None, **kwargs)
        self._completion_status = None

Our *controller* could take some time to signal that it is finished (seconds to minutes, at least).  We need a way to detect this completion.  We can do that either by polling the PV or by subscribing to a callback on the completion event.  We'll make that choice when we implement the actual activity.  Here, intend to wait in a polling loop.  Since the polling loop is an activity that does not return until the *controller* is done, we must do that waiting in a thread (consider an [alternative suggestion to use `epics.ca.CAThread`](https://github.com/NSLS-II/tutorial-docker/pull/7#discussion_r184905960)) separate from that of the RunEngine.  (We do not want to block the RunEngine thread so it is free to respond to other activities, such as data from other streams or the user inerface.)  So, we run `my_activity()` in a separate thread that is called from `kickoff()`:

        thread = threading.Thread(target=self.my_activity, daemon=True)
        thread.start()

The basic outline of `my_activity()` is:

    def my_activity(self):
        # tell the *controller* to do its work
        # wait for for it to tell us it is done
        self._completion_status._finished(success=True)

The waiting step will *block the thread* in which `my_activity()` is running but that's OK since this is separate from the RunEngine's thread.

Within `my_activity()`,  we've left two comments starting with `# TODO: ` marking where we need to add code that will complete the specifics of our final project.  Since this tutorial develops a starting template for that project (and others), we leave these comments as-is.

We've also added some diagnostic reporting (calls to `logger.info(...)`) to build out the next example:

In [4]:
class MyFlyer(ophyd.Device):
    """
    starting template for a Flyer that we understand
    """

    def __init__(self, *args, **kwargs):
        super().__init__('', parent=None, **kwargs)
        self._completion_status = None

    def my_activity(self):
        """
        start the "fly scan" here, could wait for completion
        
        It's OK to use blocking calls here 
        since this is called in a separate thread
        from the BlueSky RunEngine.
        """
        logger.info("activity()")
        if self._completion_status is None:
            logger.info("leaving activity() - not complete")
            return
        
        # TODO: do the activity here
        # TODO: wait for completion
        
        self._completion_status._finished(success=True)
        logger.info("activity() complete. status = " + str(self._completion_status))

    def kickoff(self):
        """
        Start this Flyer
        """
        logger.info("kickoff()")
        kickoff_status = ophyd.DeviceStatus(self)
        self._completion_status = ophyd.DeviceStatus(self)
        
        thread = threading.Thread(target=self.my_activity, daemon=True)
        thread.start()

        kickoff_status._finished(success=True)
        return kickoff_status

    def complete(self):
        """
        Wait for flying to be complete
        """
        logger.info("complete()")
        if self._completion_status is None:
            raise RuntimeError("No collection in progress")

        return self._completion_status

    def collect(self):
        """
        Start this Flyer
        """
        logger.info("collect()")
        self._completion_status = None
        yield {'data':{}, 'timestamps':{}, 'time':time.time()}
    
    def describe_collect(self):
        """
        Describe details for ``collect()`` method
        """
        logger.info("describe_collect()")
        return {self.name: {}}


In [5]:
ifly = MyFlyer(name="ifly")

### Diagnostics  <a class="anchor" id="Diagnostics" />

When building a `Flyer`, it is useful to have some diagnostics in place.  Already, we have been using some of these, including printing interim messages via calls to `logger.info(...)`.  Another useful diagnostic step is to call each of the methods individually to make sure they are acting as expected.

1. create an instance of the `Flyer`

    flyer = MyFlyer(name="flyer")

1. verify that `kickoff()` returns a status that is "Done"

    status = flyer.kickoff()
    status.done

1. verify that `complete()` returns a status that is "Done"

    status = flyer.complete()
    status.done

1. verify that `describe_collect()` returns a dictionary

    d = flyer.describe_collect()
    d

1. verify that `collect()` returns a generator

    g = flyer.collect()
    g

1. verify that generator is a list of data dictionaries

    list(g)


Apply some of those steps here (we'll skip testing the `ifly.complete()` method when not flying since it raises a `RuntimeError` exception if data collection is not in progress):

In [6]:
ifly.describe_collect()

{'ifly': {}}

In [7]:
list(ifly.collect())

[{'data': {}, 'time': 1525108857.7332923, 'timestamps': {}}]

Now, run this fly scan:

In [8]:
RE(bluesky.plans.fly([ifly]))

('e79c163a-1201-40df-8500-6de9cbc4588c',)

In [9]:
db[-1].stream_names

['ifly']

In [10]:
db[-1].table("ifly")

""
seq_num


Still only two `documents` were emitted from the `RunEngine`

In [11]:
list(db[-1].documents())

[('start',
  {'plan_name': 'fly',
   'plan_type': 'generator',
   'scan_id': 2,
   'time': 1525108857.7523243,
   'uid': 'e79c163a-1201-40df-8500-6de9cbc4588c'}),
 ('stop',
  {'exit_status': 'success',
   'num_events': {'ifly': 1},
   'run_start': 'e79c163a-1201-40df-8500-6de9cbc4588c',
   'time': 1525108857.7792866,
   'uid': '2ea94504-fa4a-4ffe-b06b-2c2a36ede887'})]

## First working Flyer - trivial data <a class="anchor" id="trivial-data-flyer" />

See GitHub for a [summary of changes in source code](https://github.com/prjemian/ipython_mintvm/compare/062d1765023a4d9...388eb30304e51).

To collect data, we need to modify both the `collect()` *and* the `describe_collect()` methods.  BlueSky needs to know what kind of data to expect from this Flyer, so that it can generate the correct `descriptor` document.

For the *most* trivial case, we'll return a single number (`1.2345`) as the result of the first working Flyer.

In the `describe_collect()` method, we create a dictionary that describes the data to be collected:

        d = dict(
            source = "fictional",
            dtype = "number",
            shape = []
        )
        return {
            'ifly': {
                "x": d
            }
        }

Then, in the `collect()` method, add the actual data collection code:

        t = time.time()
        d = dict(
            time=t,
            data=dict(x=1.2345),
            timestamps=dict(x=t)
        )
        yield d


In [12]:
class MyFlyer(ophyd.Device):
    """
    build a Flyer that we understand
    """

    def __init__(self, *args, **kwargs):
        super().__init__('', parent=None, **kwargs)
        self._completion_status = None

    def my_activity(self):
        """
        start the "fly scan" here, could wait for completion
        
        It's OK to use blocking calls here 
        since this is called in a separate thread
        from the BlueSky RunEngine.
        """
        logger.info("activity()")
        if self._completion_status is None:
            logger.info("leaving activity() - not complete")
            return
        
        # TODO: do the activity here
        # TODO: wait for completion
        
        self._completion_status._finished(success=True)
        logger.info("activity() complete. status = " + str(self._completion_status))

    def kickoff(self):
        """
        Start this Flyer
        """
        logger.info("kickoff()")
        kickoff_status = ophyd.DeviceStatus(self)
        self._completion_status = ophyd.DeviceStatus(self)
        
        thread = threading.Thread(target=self.my_activity, daemon=True)
        thread.start()

        kickoff_status._finished(success=True)
        return kickoff_status

    def complete(self):
        """
        Wait for flying to be complete
        """
        logger.info("complete()")
        if self._completion_status is None:
            raise RuntimeError("No collection in progress")

        return self._completion_status

    def describe_collect(self):
        """
        Describe details for ``collect()`` method
        """
        logger.info("describe_collect()")
        d = dict(
            source = "fictional",
            dtype = "number",
            shape = []
        )
        return {
            'ifly': {
                "x": d
            }
        }

    def collect(self):
        """
        Start this Flyer
        """
        logger.info("collect()")
        self._completion_status = None
        t = time.time()
        d = dict(
            time=t,
            data=dict(x=1.2345),
            timestamps=dict(x=t)
        )
        yield d

As before, create a new instance of the *revised* `MyFlyer` class.

In [13]:
ifly = MyFlyer(name="ifly")

In [14]:
print('output from describe_collect() : ', ifly.describe_collect())
print("list output from collect() : ", list(ifly.collect()))

output from describe_collect() :  {'ifly': {'x': {'source': 'fictional', 'dtype': 'number', 'shape': []}}}
list output from collect() :  [{'time': 1525108858.013278, 'data': {'x': 1.2345}, 'timestamps': {'x': 1525108858.013278}}]


Running this flyer with the RunEngine seems anticlimactic but the lack of exceptions tells us that it ran and we get a UUID at the end.

In [15]:
RE(bluesky.plans.fly([ifly]))

('60a898c8-3708-4ddc-9441-ed988afd9451',)

We next query the last scan in the databroker and show a table of the stream from `collect()`:

In [16]:
h = db[-1]
h.table(h.stream_names[0])

,time,x
seq_num,,
1,2018-04-30 13:20:58.049434,1.2345


Now, there are two more document types emitted: `descriptor` and `event`.  Only one `event` since there is only one data point.

In [17]:
list(db[-1].documents())

[('start',
  {'plan_name': 'fly',
   'plan_type': 'generator',
   'scan_id': 3,
   'time': 1525108858.029332,
   'uid': '60a898c8-3708-4ddc-9441-ed988afd9451'}),
 ('descriptor',
  {'data_keys': {'x': {'dtype': 'number', 'shape': [], 'source': 'fictional'}},
   'hints': {},
   'name': 'ifly',
   'object_keys': {'ifly': ['x']},
   'run_start': '60a898c8-3708-4ddc-9441-ed988afd9451',
   'time': 1525108858.0394192,
   'uid': 'ddcca0bc-2b83-47a2-9bf2-10d25f9febab'}),
 ('event',
  {'data': {'x': 1.2345},
   'descriptor': 'ddcca0bc-2b83-47a2-9bf2-10d25f9febab',
   'filled': {},
   'seq_num': 1,
   'time': 1525108858.0494342,
   'timestamps': {'x': 1525108858.0494342},
   'uid': '8daf91bb-1043-48c7-8ffe-16e36f9f61b8'}),
 ('stop',
  {'exit_status': 'success',
   'num_events': {'ifly': 1},
   'run_start': '60a898c8-3708-4ddc-9441-ed988afd9451',
   'time': 1525108858.06146,
   'uid': 'cbd41427-3edf-4643-8e91-369d951a2623'})]

## Flyer that "collects" 1-D array data  <a class="anchor" id="simple-1d-array-flyer" />

We continue to expand the capabilities of our working example of a Flyer.  Now, we wish to "collect" a 1-D array of data.  We'll manufacture that data ourselves to keep the code simple.

Here, we generate an array of 5 numbers, each representing the elapsed time (in seconds) since the scan started (in `kickoff()`).  We record the start time with `self.t0 = time.time()`.  Also, `time.time()` is used to generate the timestamps in the data events.  In a real case, we *might* get timestamps from the *controller* or we'd have to make it up ourselves, just like this example.

See GitHub for a [summary of changes in source code](https://github.com/prjemian/ipython_mintvm/compare/388eb30304e51...a0af3ec57a3430e777b3).

In [18]:
class MyFlyer(ophyd.Device):
    """
    a Flyer that we understand that reports 1-D array of data
    """

    def __init__(self, *args, **kwargs):
        super().__init__('', parent=None, **kwargs)
        self._completion_status = None
        self.t0 = 0

    def my_activity(self):
        """
        start the "fly scan" here, could wait for completion
        
        It's OK to use blocking calls here 
        since this is called in a separate thread
        from the BlueSky RunEngine.
        """
        logger.info("activity()")
        if self._completion_status is None:
            logger.info("leaving activity() - not complete")
            return
        
        # TODO: do the activity here
        # TODO: wait for completion
        
        self._completion_status._finished(success=True)
        logger.info("activity() complete. status = " + str(self._completion_status))

    def kickoff(self):
        """
        Start this Flyer
        """
        logger.info("kickoff()")
        kickoff_status = ophyd.DeviceStatus(self)
        self._completion_status = ophyd.DeviceStatus(self)
        self.t0 = time.time()
        
        thread = threading.Thread(target=self.my_activity, daemon=True)
        thread.start()

        kickoff_status._finished(success=True)
        return kickoff_status

    def complete(self):
        """
        Wait for flying to be complete
        """
        logger.info("complete()")
        if self._completion_status is None:
            raise RuntimeError("No collection in progress")

        return self._completion_status

    def describe_collect(self):
        """
        Describe details for ``collect()`` method
        """
        logger.info("describe_collect()")
        d = dict(
            source = "elapsed time, s",
            dtype = "number",
            shape = (1,)
        )
        return {
            self.name: {
                "x": d
            }
        }

    def collect(self):
        """
        Start this Flyer
        """
        logger.info("collect()")
        self._completion_status = None
        for _ in range(5):
            t = time.time()
            x = t - self.t0 # data is elapsed time since kickoff()
            d = dict(
                time=t,
                data=dict(x=x),
                timestamps=dict(x=t)
            )
            yield d


In [19]:
ifly = MyFlyer(name="ifly")

In [20]:
print(ifly.describe_collect())

{'ifly': {'x': {'source': 'elapsed time, s', 'dtype': 'number', 'shape': (1,)}}}


In [21]:
list(ifly.collect())

[{'data': {'x': 1525108858.26954},
  'time': 1525108858.26954,
  'timestamps': {'x': 1525108858.26954}},
 {'data': {'x': 1525108858.269543},
  'time': 1525108858.269543,
  'timestamps': {'x': 1525108858.269543}},
 {'data': {'x': 1525108858.2695444},
  'time': 1525108858.2695444,
  'timestamps': {'x': 1525108858.2695444}},
 {'data': {'x': 1525108858.2695463},
  'time': 1525108858.2695463,
  'timestamps': {'x': 1525108858.2695463}},
 {'data': {'x': 1525108858.2695477},
  'time': 1525108858.2695477,
  'timestamps': {'x': 1525108858.2695477}}]

Again, not much information from running this flyer, except that it succeeds and a uuid is returned.

In [22]:
RE(bluesky.plans.fly([ifly]))

('de14d94d-4f3b-4f40-be86-c84b724ce342',)

More information is obtained by asking the databroker about the most recent scan.  Here's the data table from our stream of data events.  (The stream is named "ifly" as set from `self.name` in the `describe_collect()` method.)  We'll reference that last scan as `h` to reduce the amount of typing.

In [23]:
h = db[-1]
h.stream_names[0]

'ifly'

In [24]:
h.table(h.stream_names[0])

,time,x
seq_num,,
1,2018-04-30 13:20:58.336185,0.016641
2,2018-04-30 13:20:58.336217,0.016673
3,2018-04-30 13:20:58.336232,0.016688
4,2018-04-30 13:20:58.336246,0.016702
5,2018-04-30 13:20:58.336260,0.016716


We generated 5 data points, there are 5 `event` documents to match.

In [25]:
list(db[-1].documents())

[('start',
  {'plan_name': 'fly',
   'plan_type': 'generator',
   'scan_id': 4,
   'time': 1525108858.3017147,
   'uid': 'de14d94d-4f3b-4f40-be86-c84b724ce342'}),
 ('descriptor',
  {'data_keys': {'x': {'dtype': 'number',
     'shape': (1,),
     'source': 'elapsed time, s'}},
   'hints': {},
   'name': 'ifly',
   'object_keys': {'ifly': ['x']},
   'run_start': 'de14d94d-4f3b-4f40-be86-c84b724ce342',
   'time': 1525108858.3202293,
   'uid': 'bd1fa7c5-23d6-4c12-9a43-c540f12b53bb'}),
 ('event',
  {'data': {'x': 0.01664137840270996},
   'descriptor': 'bd1fa7c5-23d6-4c12-9a43-c540f12b53bb',
   'filled': {},
   'seq_num': 1,
   'time': 1525108858.3361855,
   'timestamps': {'x': 1525108858.3361855},
   'uid': 'c9b4bca8-79c8-43d1-b4fa-0df41e40c4ad'}),
 ('event',
  {'data': {'x': 0.016672849655151367},
   'descriptor': 'bd1fa7c5-23d6-4c12-9a43-c540f12b53bb',
   'filled': {},
   'seq_num': 2,
   'time': 1525108858.336217,
   'timestamps': {'x': 1525108858.336217},
   'uid': 'ccfd30ea-b093-4ec6-9

## Conclusion

Now you have seen the basic steps in creating a `Flyer`.  You have also been shown a few diagnostic tools to help you investigate your code development.